In [4]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import psycopg2
import datetime as dt
from tabulate import tabulate
pd.set_option("display.max_columns", None)

### INDICE DE CANASTA BÁSICA A PARTIR DE LOS PRECIOS DE LOS PRODUCTOS EN COTO DIGITAL.

El listado de productos fue obtenido de la pagina del INDEC https://www.indec.gob.ar/ftp/cuadros/sociedad/EPH_metodologia_22_pobreza.pdf y los valores se obtienen de la pagina de Coto Digital https://www.cotodigital3.com.ar/ 
Se utiliza coto digital porque es la cadena de supermercados que tiene mayor cobertura en el país. Podriamos utilizar cualquier otra cadena de supermercados que cuenten con precios publicados en internet.

La idea es tener una referencia de la variacion de los precios de la canasta basica para poder constatarlos contra los datos del INDEC que se publican mensualmente. 
Es una recoleccion muy sencilla de datos sin tener en cuenta ubicacion geoestadistica, por lo que no es una muestra representativa de la poblacion.

Funciones para obtener los productos de la canasta basica de Coto Digital y almacenarlos en un archivo en un diccionario.

In [2]:
listado = {'fecha': dt.datetime.now().strftime("%Y-%m-%d")}

def kilo(nombre_producto, producto_url, porcion = 1):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='unit')
    valor = valor[0].get_text()

    match = re.search(r"\$([\d,.]+)", valor)
    
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        nombre = nombre_producto.replace(" ", "_")
        print(nombre, (number * porcion)) 
        listado[nombre] = (number * porcion)
        
    else:
        print("No se encontró un número en el string")

def unidad(nombre_producto, producto_url):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='atg_store_newPrice')
    valor = valor[0].get_text()
    
    match = re.search(r"\$([\d,.]+)", valor)
    
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        nombre = nombre_producto.replace(" ", "_")
        print(nombre, number)
        listado[nombre] = number
        
    else:
        print("No se encontró un número en el string")





Carga CSV con enlaces a los productos de la canasta basica de Coto Digital

In [3]:
canasta_basica = pd.read_csv('listado canasta basica.csv', sep=';', encoding='latin-1', usecols=['producto', 'cantidad_g_ml', 'url_coto', 'tipo_producto', 'porcion'])
canasta_basica.head(10)

,producto,cantidad_g_ml,url_coto,tipo_producto,porcion
0,Pan Frances,"6,75",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
1,Galletitas dulces,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
2,Galletitas de agua,420,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
3,Harina de trigo,"1,08",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
4,Harina de maiz,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,0.25
5,Arroz,"1,2",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
6,Fideos secos,"1,74",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
7,Asado,"1,23",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
8,Carnaza comun,510,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.50
9,Hueso con carne,390,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.39


Leemos cada enlace y mediante las funciones de web scraping obtenemos los precios de los productos y lo guardamos en un diccionario.

In [4]:

for producto in canasta_basica.index:
  
   if canasta_basica.loc[producto, 'tipo_producto'] == 'kilo':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      kilo(canasta_basica.loc[producto, 'producto'], url)
      
   elif canasta_basica.loc[producto, 'tipo_producto'] == 'unidad':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      unidad(canasta_basica.loc[producto, 'producto'], url)
      
   else:
      canasta_basica.loc[producto, 'url_coto'] == 'nan'
      print(producto, 'no hay url')

   


Pan_Frances 995.14
Galletitas_dulces 192.49
Galletitas_de_agua 339.49
Harina_de_trigo 201.7
Harina_de_maiz 221.92
Arroz 199.0
Fideos_secos 344.48
Asado 1599.9
Carnaza_comun 1049.9
Hueso_con_carne 529.9
Paleta 1499.9
Carne_picada 999.9
Nalga 1749.9
Higado 299.9
Pechito_de_cerdo 999.9
Pollo 415.9
Carne_de_pescado 1590.0
Mortadela 1185.0
Paleta_cocida 1850.0
Salchich¢n 1285.0
Salame 2660.0
Aceite_de_girasol 556.92
Margarina_para_cocinar 268.45
Leche_fluida 234.66
Leche_en_polvo_entera 1439.49
Queso_crema 256.01
Queso_cuartirolo 1399.0
Queso_de_rallar 2579.0
Manteca 1542.75
Yogur 218.0
Dulce_de_leche 713.37
Huevo 1053.0
Manzana 449.0
33 no hay url
Naranja 399.0
Banana 389.0
Pera 279.0
Batata 359.0
Papa 319.0
Acelga 239.0
Cebolla 339.0
Choclo 199.0
Lechuga 799.0
Tomate_perita 399.0
Zanahoria 199.0
Zapallo 189.0
Tomate_envasado 595.12
Arvejas_en_lata 458.04
Lentejas_secas 1085.25
Azucar 273.0
Dulce_de_batata 726.0
Mermelada 563.23
Sal_fina 306.5
Mayonesa 620.63
Vinagre 174.03
Caldo_concentra

### Cargar todo a la base de datos postgres

Creo el diccionario de productos para la creacion de la tabla. 

In [5]:
tabla = {'id' : 'SERIAL PRIMARY KEY', 'fecha' : 'DATE' }

for producto in listado:
    if producto == 'fecha':
        continue
    tabla[producto] = 'FLOAT'

tabla

{'id': 'SERIAL PRIMARY KEY',
 'fecha': 'DATE',
 'Pan_Frances': 'FLOAT',
 'Galletitas_dulces': 'FLOAT',
 'Galletitas_de_agua': 'FLOAT',
 'Harina_de_trigo': 'FLOAT',
 'Harina_de_maiz': 'FLOAT',
 'Arroz': 'FLOAT',
 'Fideos_secos': 'FLOAT',
 'Asado': 'FLOAT',
 'Carnaza_comun': 'FLOAT',
 'Hueso_con_carne': 'FLOAT',
 'Paleta': 'FLOAT',
 'Carne_picada': 'FLOAT',
 'Nalga': 'FLOAT',
 'Higado': 'FLOAT',
 'Pechito_de_cerdo': 'FLOAT',
 'Pollo': 'FLOAT',
 'Carne_de_pescado': 'FLOAT',
 'Mortadela': 'FLOAT',
 'Paleta_cocida': 'FLOAT',
 'Salchich¢n': 'FLOAT',
 'Salame': 'FLOAT',
 'Aceite_de_girasol': 'FLOAT',
 'Margarina_para_cocinar': 'FLOAT',
 'Leche_fluida': 'FLOAT',
 'Leche_en_polvo_entera': 'FLOAT',
 'Queso_crema': 'FLOAT',
 'Queso_cuartirolo': 'FLOAT',
 'Queso_de_rallar': 'FLOAT',
 'Manteca': 'FLOAT',
 'Yogur': 'FLOAT',
 'Dulce_de_leche': 'FLOAT',
 'Huevo': 'FLOAT',
 'Manzana': 'FLOAT',
 'Naranja': 'FLOAT',
 'Banana': 'FLOAT',
 'Pera': 'FLOAT',
 'Batata': 'FLOAT',
 'Papa': 'FLOAT',
 'Acelga': 'F

Inserto los precios de los productos en la base de datos

In [6]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="variacion",
    user="postgres",
    password="postgres"
)

# Insertar los datos en la tabla
with conn.cursor() as cur:
    columnas = []
    valores = []
    for columna, valor in listado.items():
        columnas.append(columna)
        valores.append(valor)
        
    data_fecha = cur.execute("SELECT fecha FROM precios WHERE fecha = %s", (valores[0],))
    data_fecha = cur.fetchone()
    print(data_fecha)
    if data_fecha == None:
        print('no hay fecha')
        print('Cargando datos') 
        query = f"INSERT INTO precios ({', '.join(columnas)}) VALUES ({', '.join(['%s'] * len(valores))})"
        cur.execute(query, valores)
        conn.commit()
    else:
        print('Ya se cargaron los datos de hoy')




# Cerrar la conexión a la base de datos
cur.close()
conn.close()

(datetime.date(2023, 2, 27),)
Ya se cargaron los datos de hoy


Traer los productos de la canasta basica desde la tabla de postgres para almacenarlos en CSV o EXCEL. Formatos alternativos para el archivo de salida.

In [7]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="variacion",
    user="postgres",
    password="postgres"
)

# Obtener los datos de la tabla
with conn.cursor() as cur:
    cur.execute("SELECT * FROM precios")
    rows = cur.fetchall()
    conn.commit()

# crear un dataframe con los datos
df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])



#Cerrar la conexión a la base de datos
cur.close()
conn.close()

# Guardar el dataframe en un archivo csv
df.to_csv('./datos/precios.csv', index=False)

# guardar el dataframe en un archivo excel
df.to_excel('./datos/precios.xlsx', index=False)

df.head(10)

,id,fecha,pan_frances,galletitas_dulces,galletitas_de_agua,harina_de_trigo,harina_de_maiz,arroz,fideos_secos,asado,carnaza_comun,hueso_con_carne,paleta,carne_picada,nalga,higado,pechito_de_cerdo,pollo,carne_de_pescado,mortadela,paleta_cocida,salchich¢n,salame,aceite_de_girasol,margarina_para_cocinar,leche_fluida,leche_en_polvo_entera,queso_crema,queso_cuartirolo,queso_de_rallar,manteca,yogur,dulce_de_leche,huevo,manzana,naranja,banana,pera,batata,papa,acelga,cebolla,choclo,lechuga,tomate_perita,zanahoria,zapallo,tomate_envasado,arvejas_en_lata,lentejas_secas,azucar,dulce_de_batata,mermelada,sal_fina,mayonesa,vinagre,caldo_concentrado,gaseosas,jugos_concentrados,soda,cerveza,vino,cafe,yerba,te_en_saquitos
0,1,2023-02-17,995.14,192.49,339.49,201.75,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,1349.9,1749.9,299.9,999.9,415.9,1499.0,1185.0,1850.0,1285.0,2660.0,556.92,268.45,215.28,1439.49,234.87,1399.0,2579.0,1542.75,218.0,527.02,1053.0,499.0,369.0,299.0,279.0,359.0,239.0,239.0,299.0,169.0,799.0,399.0,199.0,189.0,595.12,458.04,857.92,273.0,726.0,563.23,306.5,620.63,148.47,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
1,11,2023-02-18,995.14,192.49,339.49,201.75,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,1349.9,1749.9,299.9,999.9,415.9,1499.0,1185.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,1399.0,2579.0,1542.75,218.0,574.45,1053.0,449.0,399.0,299.0,279.0,359.0,239.0,239.0,299.0,169.0,749.0,349.0,199.0,189.0,595.12,458.04,986.60,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
2,14,2023-02-20,995.14,192.49,339.49,201.75,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1499.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,574.45,1053.0,449.0,399.0,299.0,279.0,299.0,319.0,239.0,299.0,199.0,749.0,349.0,199.0,189.0,595.12,458.04,986.60,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
3,15,2023-02-21,995.14,192.49,339.49,201.75,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1499.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,574.45,1053.0,449.0,399.0,299.0,279.0,299.0,319.0,239.0,299.0,199.0,749.0,349.0,199.0,189.0,595.12,458.04,986.60,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
4,16,2023-02-22,995.14,192.49,339.49,201.75,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1499.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,574.45,1053.0,449.0,399.0,389.0,279.0,299.0,319.0,239.0,299.0,199.0,749.0,349.0,199.0,189.0,595.12,458.04,986.60,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
5,17,2023-02-23,995.14,192.49,339.49,201.70,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1499.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,713.37,1053.0,449.0,399.0,389.0,279.0,299.0,239.0,239.0,229.0,199.0,799.0,349.0,249.0,189.0,595.12,458.04,1085.25,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
6,18,2023-02-24,995.14,192.49,339.49,201.70,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1590.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,713.37,1053.0,449.0,399.0,389.0,279.0,299.0,239.0,239.0,229.0,249.0,799.0,399.0,279.0,189.0,595.12,458.04,1085.25,273.0,726.0,563.23,306.5,620.63,174.03,935.87,87.04,217.73,104.48,161.64,440.0,830.58,770.85,418.56
7,19,2023-02-25,995.14,192.49,339.49,201.70,221.92,199.0,344.48,1599.9,1049.9,529.9,1499.9,999.9,1749.9,299.9,999.9,415.9,1590.0,1480.0,1850.0,1285.0,2660.0,556.92,268.45,234.66,1439.49,256.01,979.0,2579.0,1542.75,218.0,713.37,1053.0,449.0,399.0,389.

Hacemos una variacion de precios de los productos de la canasta basica para poder compararlos con los datos del INDEC.

In [8]:
listado_precios = pd.read_csv('./datos/precios.csv', sep=',', encoding='latin-1')
lista_variaciones = {}
for producto in listado_precios:
    if producto == 'fecha':
        continue
    elif producto == 'id':
        continue
    else:
        variacion = (listado_precios[producto].iloc[-1] / listado_precios[producto].iloc[0] - 1) * 100
        lista_variaciones[producto] = variacion
        
precio_anterior = sum(listado_precios.iloc[0, 2:])
precio_actual = sum(listado_precios.iloc[-1, 2:])
print(f"El precio anterior es ${precio_anterior}")
print(f"El precio actual es ${precio_actual}")
suma = sum(lista_variaciones.values()) / len(lista_variaciones)
print('La variación de precios de la canasta básica es del', round(suma, 2), '%')


El precio anterior es $43354.810000000005
El precio actual es $43795.53000000001
La variación de precios de la canasta básica es del 2.7 %


### Creamos un csv en formato de lista larga para poder usarlo correctamente en data studio.

In [9]:
lista_tabla = df.columns.to_list()

lista_tabla.remove('id')
lista_tabla.remove('fecha')
lista_tabla

tupla = tuple(lista_tabla)


In [10]:
lista_larga = df.melt(id_vars=['fecha'], value_vars=lista_tabla, var_name='producto', value_name='precio')
lista_larga.head(10)

,fecha,producto,precio
0,2023-02-17,pan_frances,995.14
1,2023-02-18,pan_frances,995.14
2,2023-02-20,pan_frances,995.14
3,2023-02-21,pan_frances,995.14
4,2023-02-22,pan_frances,995.14
5,2023-02-23,pan_frances,995.14
6,2023-02-24,pan_frances,995.14
7,2023-02-25,pan_frances,995.14
8,2023-02-26,pan_frances,995.14
9,2023-02-27,pan_frances,995.14


In [19]:
lista_larga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     630 non-null    object 
 1   producto  630 non-null    object 
 2   precio    630 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.9+ KB


In [11]:
lista_larga.to_csv('./datos/precios_lista_larga.csv', index=False)

### Subimos la lista_larga a una DDBB en postgres online. 
 

la base esta en NEON.tech

In [2]:
from certificados_ddbb import ddbb_pass

In [25]:


# Conexión a la base de datos
conn = psycopg2.connect(
    host="ep-jolly-glitter-302666.us-east-2.aws.neon.tech",
    database="neondb",
    user="Vosinepi",
    password=ddbb_pass
)

# Crear la tabla si no existe
with conn.cursor() as cur:
    
    cur.execute(f"CREATE TABLE IF NOT EXISTS precios_lista_larga (fecha DATE, producto VARCHAR(255), precio FLOAT)")
    conn.commit()

# Insertar los datos en la tabla



    for i in lista_larga.index:
        producto_fecha = cur.execute("SELECT fecha, producto FROM precios_lista_larga WHERE fecha = %s AND producto = %s", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto']))
        producto_fecha = cur.fetchone()
        
        if producto_fecha == None:
            print(f'no existe, cargando {lista_larga.loc[i, "producto"]}')
            
            cur.execute("INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto'], lista_larga.loc[i, 'precio']))
            conn.commit()
        elif (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto']) in producto_fecha:
            print(f'ya existe {lista_larga.loc[i, "producto"]}')
            continue


# with conn.cursor() as cur:
#     for i in lista_larga.index:
#         cur.execute("INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto'], lista_larga.loc[i, 'precio']))
#         conn.commit()



#Cerrar la conexión a la base de datos
cur.close()
conn.close()

print('Datos cargados')
    

(datetime.date(2023, 2, 17), 'pan_frances')
(datetime.date(2023, 2, 18), 'pan_frances')
(datetime.date(2023, 2, 20), 'pan_frances')
(datetime.date(2023, 2, 21), 'pan_frances')
(datetime.date(2023, 2, 22), 'pan_frances')
(datetime.date(2023, 2, 23), 'pan_frances')
(datetime.date(2023, 2, 24), 'pan_frances')
(datetime.date(2023, 2, 25), 'pan_frances')
(datetime.date(2023, 2, 26), 'pan_frances')
(datetime.date(2023, 2, 27), 'pan_frances')
(datetime.date(2023, 2, 17), 'galletitas_dulces')
(datetime.date(2023, 2, 18), 'galletitas_dulces')
(datetime.date(2023, 2, 20), 'galletitas_dulces')
(datetime.date(2023, 2, 21), 'galletitas_dulces')
(datetime.date(2023, 2, 22), 'galletitas_dulces')
(datetime.date(2023, 2, 23), 'galletitas_dulces')
(datetime.date(2023, 2, 24), 'galletitas_dulces')
None
no existe, cargando galletitas_dulces
None
no existe, cargando galletitas_dulces
None
no existe, cargando galletitas_dulces
None
no existe, cargando galletitas_de_agua
None
no existe, cargando galletitas_d